<a href="https://colab.research.google.com/github/tymoshenkoanna/memo_parsing/blob/main/memo_parsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from urllib.request import urlopen
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs4

# 1. create a list of links from one url
core_url = 'https://ru.openlist.wiki/'
search_url = 'https://ru.openlist.wiki/%D0%A1%D0%BB%D1%83%D0%B6%D0%B5%D0%B1%D0%BD%D0%B0%D1%8F:OlSearch?olsearch-name=*&olsearch-birth_min=&olsearch-birth_max=&olsearch-death_min=&olsearch-death_max=&olsearch-birthplace=&olsearch-liveplace=&olsearch-nationality=&olsearch-social=&olsearch-profession=&olsearch-deathplace=&olsearch-burialplace=&olsearch-body=&olsearch-categories=&olsearch-arrest_min=&olsearch-arrest_max=&olsearch-indictment=&olsearch-conviction_min=&olsearch-conviction_max=&olsearch-conviction-org=&olsearch-sentence=&olsearch-detentionplace=&olsearch-release_min=&olsearch-release_max=&olsearch-execution_min=&olsearch-execution_max=&olsearch-archive-case-number=&olsearch-run=1&olsearch-advform=1#OlSearch-results-caption'
reqs = requests.get(search_url)
bsl = bs4(reqs.text, 'html.parser')

#extract all links from the search_url, searching by <a> tag
links_http = open('links_http.txt', 'w')
for link in bsl.find_all('a'):
    links_http.write("%s\n" % link)

#remove <a href=... at the beginning of each link line
with open('links_http.txt') as oldfile, open('links_http_cut.txt', 'w') as newfile:
    for line in oldfile:
         if ':' not in line:
            sep_1 = '"/'
            cut_1 = line.split(sep_1, 1)[1]
            newfile.write(cut_1)

#remove end of each line after > sign, to leave only actual link         
with open('links_http_cut.txt') as oldfile_2, open('links_final.txt', 'w') as newfile_2:
  for line in oldfile_2:
      if '>' in line and ')' in line:
            sep_2 = '">'
            cut_2 = line.split(sep_2, 1)[0]
            newfile_2.write("https://ru.openlist.wiki/%s\n" % cut_2)

f = open('links_final.txt', 'r')
links_file = f.read()
#print (links_file)

#2. creating headers for the table
source = requests.get('https://ru.openlist.wiki/H%D0%B8%D0%BA%D0%B8%D1%82%D1%8E%D0%BA_%D0%92%D0%B0%D0%BB%D0%B5%D1%80%D0%B8%D0%B9_%D0%90%D0%BB%D0%B5%D0%BA%D1%81%D0%B0%D0%BD%D0%B4%D1%80%D0%BE%D0%B2%D0%B8%D1%87_(1912)')  
memo_data = bs4(source.text, 'lxml') #creates an object with all html data from the page

column_names = []
for tag_li in memo_data.find_all("li"):#extract all lines that start with <li> tag
  for tag_b in tag_li.find_all("b"):
    pr_data = ("{0}: {1}".format(tag_li.name, tag_li.text))#extract all lines that include <b> tag
    for line in pr_data:
      sep_3 = ':  '#define separator by which line will be split --> ":  " colon and two spaces
      pr_data_notag = pr_data.split(sep_3, 1)[1]#defined new cleaned data
      f_data_1 = pr_data_notag.replace(": ",":") #remove not needed spaces afetr ":"
      final_data = f_data_1.replace(", ",",") #remove not needed spaces after ","
      sep_table_header = ':'
      table_header = final_data.split(sep_table_header, 1)[0]
    column_names.append(table_header)

#print(column_names) printing column names

#3. extracting values and adding them to the table with column names, which were extracted in step 2 above
person_data = []
for tag_li in memo_data.find_all("li"):#extract all lines that start with <li> tag
  for tag_b in tag_li.find_all("b"):
    pr_data = ("{0}: {1}".format(tag_li.name, tag_li.text))#extract all lines that include <b> tag
    for line in pr_data:
      sep_3 = ':  '#define separator by which line will be split --> ":  " colon and two spaces
      pr_data_notag = pr_data.split(sep_3, 1)[1]#defined new cleaned data
      f_data_1 = pr_data_notag.replace(": ",":") #remove not needed spaces afetr ":"
      final_data = f_data_1.replace(", ",",") #remove not needed spaces after ","
      sep_table_data = ':'
      p_data = final_data.split(sep_table_data, 1)[1]
    person_data.append(p_data)
  
#print(person_data) printing all data

#4. Create a table
import numpy as np
from tabulate import tabulate
data= [person_data]
print(tabulate(data, headers=column_names))

Дата рождения    Место рождения                Пол      Национальность    Гражданство (подданство)    Профессия / место работы    Место проживания                      Где и кем арестован     Дата ареста         Обвинение              Осуждение          Осудивший орган                Статья          Приговор    Дата реабилитации    Реабилитирующий орган          Комментарий к аресту                                  Источники данных
---------------  ----------------------------  -------  ----------------  --------------------------  --------------------------  ------------------------------------  ----------------------  ------------------  ---------------------  -----------------  -----------------------------  --------------  ----------  -------------------  -----------------------------  ----------------------------------------------------  ----------------------------------------------------------------------------------------------
1912 г.          ГССР,Аджарская АССР,г.Батуми  муж